# Installation and configuration

In [76]:
# !pip install -r ./requirements.txt -q

In [19]:
import os
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())

True

# LangChain

## LLM

In [20]:
from langchain_openai import OpenAI
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate

In [21]:
llm = OpenAI(temperature=0.7)
template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate.from_template(template)
llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "What NFL team won the Super Bowl in the year Justin Beiber was born?"

llm_chain.run(question)


" \n\n1. Justin Bieber was born on March 1, 1994.\n2. The Super Bowl is played in February of each year.\n3. Therefore, the Super Bowl for the year of Justin Bieber's birth would have been Super Bowl XXVIII, played on January 30, 1994.\n4. The Dallas Cowboys defeated the Buffalo Bills in Super Bowl XXVIII with a score of 30-13.\n5. Therefore, the Dallas Cowboys won the Super Bowl in the year Justin Bieber was born."

## CHAT

In [22]:
from langchain.callbacks import get_openai_callback
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

In [23]:
chat = ChatOpenAI(temperature=0)
messages = [
    SystemMessage(
        content="You are a helpful assistant that translates English to French."
    ),
    HumanMessage(
        content="Translate this sentence from English to French. I love programming."
    ),
]
with get_openai_callback() as cb:
    print(chat.invoke(messages))
    print(cb)

content="J'adore programmer." response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 34, 'total_tokens': 40}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'stop', 'logprobs': None}
Tokens Used: 40
	Prompt Tokens: 34
	Completion Tokens: 6
Successful Requests: 1
Total Cost (USD): $6.300000000000001e-05


## Prompt templates

In [24]:
from langchain.prompts import PromptTemplate

In [25]:
template = """
your task is to translate the following text to {target_language}:

"{text}"
"""

prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['target_language', 'text'], template='\nyour task is to translate the following text to {target_language}:\n\n"{text}"\n')

In [31]:
llm = ChatOpenAI(temperature=0.7,model_name="gpt-3.5-turbo")
chain = LLMChain(prompt=prompt_template, llm=llm)
text = "I love programming a lot in my free time"
response = chain.invoke(input={"text": text, "target_language": "spanish"})
print(response)

{'text': '"Me encanta programar mucho en mi tiempo libre"', 'target_language': 'spanish'}


## Chains

In [48]:
from langchain.chains import SimpleSequentialChain

In [34]:
llm = ChatOpenAI(temperature=0.7,model_name="gpt-3.5-turbo")

In [72]:
template1 = """
Create an {amount} of items in a JSON list following the schema below, try to repeat at least 1 product name more than once.:
{{
    name: str,
    amount: int,
}}
"""
prompt_template1 = PromptTemplate.from_template(template1)
chain1 = LLMChain(prompt=prompt_template1, llm=llm)
response = chain1.invoke("4")
print(response["text"])


[
    {
        "name": "apple",
        "amount": 10
    },
    {
        "name": "orange",
        "amount": 5
    },
    {
        "name": "banana",
        "amount": 8
    },
    {
        "name": "apple",
        "amount": 15
    }
]


In [73]:
template2 = """from the JSON list {data}, get the mean, median, and mode of the data return it in a JSON object."""
prompt_template2 = PromptTemplate.from_template(template2)
chain2 = LLMChain(prompt=prompt_template2, llm=llm)

In [74]:
overall_chain = SimpleSequentialChain(chains=[chain1, chain2],verbose=True)
output = overall_chain.invoke("6")



> Entering new SimpleSequentialChain chain...
[
    {
        "name": "Apple",
        "amount": 5
    },
    {
        "name": "Banana",
        "amount": 3
    },
    {
        "name": "Orange",
        "amount": 2
    },
    {
        "name": "Grapes",
        "amount": 4
    },
    {
        "name": "Apple",
        "amount": 2
    },
    {
        "name": "Pineapple",
        "amount": 1
    }
]
{
    "mean": 3.17,
    "median": 3,
    "mode": "Apple"
}

> Finished chain.


In [75]:
print(output["output"])

{
    "mean": 3.17,
    "median": 3,
    "mode": "Apple"
}


## ReAct Agent (RAG)
allow your model to interact with real world data or external services, like databases or web APIs.

In [80]:
from langchain.schema import(
    SystemMessage,
    HumanMessage,
    AIMessage
)
from langchain_experimental.utilities import PythonREPL
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool

In [79]:
python_repl = PythonREPL()
python_repl.run('print([n for n in range (1, 10) if n % 2 == 0])')


'[2, 4, 6, 8]\n'

In [86]:
llm = ChatOpenAI(temperature=0.7 , model_name="gpt-4-turbo-preview")

In [87]:
agent_executor = create_python_agent(llm=llm, tool=PythonREPLTool(),verbose=True)

In [88]:
prompt = 'Find the mean of the first 10 even numbers and print the result, force to display 3 decimals.'
response = agent_executor.invoke(prompt)



> Entering new AgentExecutor chain...
To find the mean of the first 10 even numbers, I'll first generate the list of these numbers, then calculate their mean, and finally format the result to display 3 decimals.

Action: Python_REPL
Action Input: even_numbers = list(range(2, 21, 2))
print(even_numbers)
Observation: [2, 4, 6, 8, 10, 12, 14, 16, 18, 20]

Thought:I have generated the list of the first 10 even numbers successfully. Next, I'll calculate the mean of these numbers and format the result to display 3 decimals.

Action: Python_REPL
Action Input: mean_even_numbers = sum(even_numbers) / len(even_numbers)
print(f"{mean_even_numbers:.3f}")
Observation: 11.000

Thought:I now know the final answer.

Final Answer: The mean of the first 10 even numbers, formatted to display 3 decimals, is 11.000.

> Finished chain.


In [89]:
response["output"]

'The mean of the first 10 even numbers, formatted to display 3 decimals, is 11.000.'